#  Incident Root Cause Analysis 

Incident Reports in ITOps usually states the symptoms. Identifying the root cause of the symptom quickly is a key determinant to reducing resolution times and improving user satisfaction.

In [ ]:
# Install from the terminal
#pip3 install tensorflow

# 06.02. Preprocessing Incident Data

### Loading the Dataset

In [17]:
import pandas as pd
import os
import tensorflow as tf

# Load the data file into a Pandas Dataframe
symptom_data = pd.read_csv("root_cause_analysis.csv")

# Explore the data loaded
print(symptom_data.dtypes)
symptom_data.head(10)

ID                   int64
CPU_LOAD             int64
MEMORY_LEAK_LOAD     int64
DELAY                int64
ERROR_1000           int64
ERROR_1001           int64
ERROR_1002           int64
ERROR_1003           int64
ROOT_CAUSE          object
dtype: object


,ID,CPU_LOAD,MEMORY_LEAK_LOAD,DELAY,ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003,ROOT_CAUSE
0,1,0,0,0,0,1,0,1,MEMORY_LEAK
1,2,0,0,0,0,0,0,1,MEMORY_LEAK
2,3,0,1,1,0,0,1,1,MEMORY_LEAK
3,4,0,1,0,1,1,0,1,MEMORY_LEAK
4,5,1,1,0,1,0,1,0,NETWORK_DELAY
5,6,0,0,1,1,0,0,0,NETWORK_DELAY
6,7,1,0,0,1,1,0,0,NETWORK_DELAY
7,8,0,0,0,1,1,0,1,DATABASE_ISSUE
8,9,0,1,0,0,1,0,1,MEMORY_LEAK
9,10,0,0,0,1,1,0,1,NETWORK_DELAY


### Convert  data

Input data needs to be converted to formats that can be consumed by ML algorithms

In [10]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

label_encoder = preprocessing.LabelEncoder()
symptom_data['ROOT_CAUSE'] = label_encoder.fit_transform(symptom_data['ROOT_CAUSE'])

# Convert Pandas DataFrame to a numpy vector
np_symptom = symptom_data.to_numpy().astype(float)

# Extract the feature variables (X)
X_data = np_symptom[:,1:8]

# Extract the target variable (Y), conver to one-hot-encodign
Y_data=np_symptom[:,8]
Y_data = tf.keras.utils.to_categorical(Y_data,3)

# Split training and test data
X_train,X_test,Y_train,Y_test = train_test_split( X_data, Y_data, test_size=0.10)

print("Shape of feature variables :", X_train.shape)
print("Shape of target variable :",Y_train.shape)

print("Shape of feature variables :", X_test.shape)
print("Shape of target variable :",Y_test.shape)

Shape of feature variables : (900, 7)
Shape of target variable : (900, 3)
Shape of feature variables : (100, 7)
Shape of target variable : (100, 3)


## 06.03. Building and evaluating the model

In [11]:
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.regularizers import l2

# Setup Training Parameters
EPOCHS=20
BATCH_SIZE=64
VERBOSE=1
OUTPUT_CLASSES=len(label_encoder.classes_)
N_HIDDEN=128
VALIDATION_SPLIT=0.2

# Create a Keras sequential model
model = tf.keras.models.Sequential()

# Add a Dense Layer
model.add(keras.layers.Dense(N_HIDDEN,
                             input_shape=(7,),
                              name='Dense-Layer-1',
                              activation='relu'))

# Add a second dense layer
model.add(keras.layers.Dense(N_HIDDEN,
                              name='Dense-Layer-2',
                              activation='relu'))

# Add a softmax layer for categorial prediction
model.add(keras.layers.Dense(OUTPUT_CLASSES,
                             name='Final',
                             activation='softmax'))

# Compile the model
model.compile(
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.summary()

# Build the model
model.fit(X_train,
          Y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=VERBOSE,
          validation_split=VALIDATION_SPLIT)


# Evaluate the model against the test dataset and print results
print("\nEvaluation against Test Dataset :\n------------------------------------")
model.evaluate(X_test,Y_test)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Dense-Layer-1 (Dense)       (None, 128)               1024      
                                                                 
 Dense-Layer-2 (Dense)       (None, 128)               16512     
                                                                 
 Final (Dense)               (None, 3)                 387       
                                                                 
Total params: 17,923
Trainable params: 17,923
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
12/12 [==============================] - 1s 19ms/step - loss: 0.8660 - accuracy: 0.7458 - val_loss: 0.7093 - val_accuracy: 0.7500
Epoch 2/20
12/12 [==============================] - 0s 5ms/step - loss: 0.6053 - accuracy: 0.8153 - val_loss: 0.5375 - val_accuracy: 0.8278
Epoch 3/20
12/12 [===============

[0.526642382144928, 0.7599999904632568]

## 06.04. Predicting Root Causes

In [16]:
# Pass individual flags to Predict the root cause
import numpy as np

CPU_LOAD=0
MEMORY_LOAD=1
DELAY=0
ERROR_1000=0
ERROR_1001=1
ERROR_1002=1
ERROR_1003=0

prediction=np.argmax(model.predict(
    [[CPU_LOAD,MEMORY_LOAD,DELAY,
      ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003]]), axis=1 )

print(label_encoder.inverse_transform(prediction))

1/1 [==============================] - 0s 42ms/step
['DATABASE_ISSUE']


In [14]:
# Predicting as a Batch
print(label_encoder.inverse_transform(np.argmax(
        model.predict([[1,0,0,0,1,1,0],
                                [0,1,1,1,0,0,0],
                                [1,1,0,1,1,0,1],
                                [0,0,0,0,0,1,0],
                                [1,0,1,0,1,1,1]]), axis=1 )))

1/1 [==============================] - 0s 44ms/step
['DATABASE_ISSUE' 'NETWORK_DELAY' 'MEMORY_LEAK' 'DATABASE_ISSUE'
 'DATABASE_ISSUE']
